<a href="https://colab.research.google.com/github/tomato-sugar/python_lesson/blob/main/magenta_basic/magenta_basic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2022年 東北TECH道場 郡山道場 第7期

##Magentaを用いた機械学習作曲入門（２）

##《このセクションで学ぶこと》
- Polyphony RNN でバッハ風合唱曲の自動作曲を試みます。
- Drums RNN でドラム演奏パートの自動作曲を試みます。
- Peformance RNN で高度なピアノ演奏の自動作曲を試みます。

##《Polyphony RNN でバッハ風合唱曲の作曲》
Polyphony RNN でバッハ風合唱曲の作曲をしてみましょう。
Polyphony RNN の詳細については、[こちら](https://github.com/magenta/magenta/tree/main/magenta/models/polyphony_rnn)を確認してください。

###(1) ライブラリのインストール
次のライブラリをインストールします。少し時間がかかります。
- Magenta：機械学習による音楽生成ライブラリ
- pyFluidSynth：音楽を生成するためのソフトウェア シンセサイザー
- pretty_midi：MIDI データを処理するためのユーティリティ

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

###(2) 基本となる音を与える

In [ ]:
from IPython.core.display import set_matplotlib_formats


# キーボードから音符を入力する関数
def base_music_create(base_music):
    start = 0.0
    total_time = 0.0
    # 音階とピッチの辞書
    scale_to_pich = {
        "ド": 60,
        "レ": 62,
        "ミ": 64,
        "ファ": 65,
        "ソ": 67,
        "ラ": 69,
        "シ": 71,
        "ド^": 72,
    }

    print("カタカナで「ド」～「ド^」を「,」区切りで入力してください。")
    print("【例】ド,レ,ミ,ド,レ,ミ,ソ,ミ,レ,ド,レ,ミ,レ")
    onpu = input("音符＞ ")
    onpu = onpu.split(",")
    for on in onpu:
        if on in scale_to_pich:
            base_music.notes.add(
                pitch=scale_to_pich[on],
                start_time=start,
                end_time=start + 0.5,
                velocity=80
            )
            start += 0.5
            total_time += 0.5

    # 所要時間の返却
    return total_time


import magenta
import note_seq
from note_seq.protobuf import music_pb2

# 基本の音を格納するbase_musicを作成
base_music = music_pb2.NoteSequence()

# 音符入力関数の呼び出し
total = base_music_create(base_music)

base_music.total_time = total  # 所要時間
base_music.tempos.add(qpm=75.0)  # 曲のテンポ（4分音符が1分間に75）

# 再生
note_seq.play_sequence(base_music, synth=note_seq.fluidsynth)  

###(3) 学習済みモデルの準備
Polyphony RNN では、学習済みモデルに「polyphony_rnn.mag」が用意されています。これを読み込み利用します。

In [ ]:
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# 学習済みモデルの初期化
note_seq.notebook_utils.download_bundle(  # 学習済みデータのダウンロード
    "polyphony_rnn.mag", "magenta/models/"
)
bundle = sequence_generator_bundle.read_bundle_file(    # 学習済みデータの読み込み
    "magenta/models/polyphony_rnn.mag"
)
generator_map = polyphony_sequence_generator.get_generator_map()  # 生成器のmapの作成
polyphony_rnn = generator_map["polyphony"](checkpoint=None, bundle=bundle)  # mapから生成器の設定
polyphony_rnn.initialize()  # 初期化

###(4) 自動作曲
上記で生成した生成器を使って作曲を行います。

In [ ]:
from note_seq.protobuf import generator_pb2

base_seq = base_music  # ベースになる音
total_sec = 60  # 生成したい曲の秒数
temperature = 1.0  # 曲のランダム度合い
base_end_time = base_music.total_time  #ベース曲の終了時刻

# 生成器のオプションの設定
options = generator_pb2.GeneratorOptions()  # 生成器のオプション
options.args["temperature"].float_value = temperature  # ランダム度合い
options.generate_sections.add(
    start_time = base_end_time,  # 作曲開始時刻
    end_time = total_sec  # 作曲終了時刻
)
# 曲の生成
gen_seq = polyphony_rnn.generate(base_seq, options)

# 再生
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

###(5) 生成した音楽データを保存

In [5]:
# MIDIデータに変換し保存
note_seq.sequence_proto_to_midi_file(
    gen_seq, "magenta/polyphony_rnn1.mid"
)  

##《Drums RNN でドラム演奏パートの作曲》
Drums RNN でドラム演奏パートの作曲をしてみましょう。 Drums RNN の詳細については、[こちら](https://github.com/magenta/magenta/tree/main/magenta/models/drums_rnn)を確認してください。なお、Drums RNN は、他のメロディー生成モデルとは異なり、使用できる音はドラムセットの用のノートナンバーに指定された9音のみです。




###(1) 基本となる音を与える
**もし、「《Polyphony RNN でバッハ風合唱曲の作曲》」を飛ばした場合には、「(1) ライブラリのインストール」を行ってから、以下を進めてください。**

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

"""
使用できる音はドラムセットの用のノートナンバーに指定された9音のみ
    36,         # C3 - Bass Drum 1
    40,         # E3 - Electric Snare
    41,         # F3 - Low Floor Tom
    42,         # Gb3/F#3 - Closed Hi-Hat
    46,         # Bb3/A#3 - Open Hi-Hat
    48,         # C4 - Hi-Mid Tom
    50,         # D4 - High Tom
    51,         # Eb4/D#4 - Ride Cymbal 1
    57,         # A4 - Crash Cymbal 2
"""

# 基本の音を格納するdrumsを作成
drums = music_pb2.NoteSequence()

# drumsに音を追加
# pitch：音の周波数  start_time：音の開始時間  end_time：終了時間  velocity：音の速さ
drums.notes.add(pitch=36, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.125, end_time=0.25, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=50, start_time=.25, end_time=0.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=41, start_time=0.375, end_time=0.5, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=38, start_time=0.5, end_time=0.625, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=.625, end_time=0.75, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=.875, end_time=1.0, is_drum=True, instrument=10, velocity=80)

drums.total_time = 1.0  # 所要時間
drums.tempos.add(qpm=120.0)  # 曲のテンポ（4分音符が1分間に120）

# 再生
note_seq.play_sequence(drums, synth=note_seq.fluidsynth)  

###(2) 学習済みモデルの準備
Drums RNN では、学習済みモデルには「drum_kit_rnn.mag」が用意されています。これを読み込み利用します。

In [ ]:
from magenta.models.drums_rnn import drums_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# 学習済みモデルの初期化
note_seq.notebook_utils.download_bundle(  # 学習済みデータのダウンロード
    "drum_kit_rnn.mag", "magenta/models/"
)
bundle = sequence_generator_bundle.read_bundle_file(    # 学習済みデータの読み込み
    "magenta/models/drum_kit_rnn.mag"
)
generator_map = drums_rnn_sequence_generator.get_generator_map()  # 生成器のmapの作成
drums_rnn = generator_map["drum_kit"](checkpoint=None, bundle=bundle)  # mapから生成器の設定
drums_rnn.initialize()  # 初期化

###(3) 自動作曲
上記で生成した生成器を使って作曲を行います。

In [ ]:
from note_seq.protobuf import generator_pb2

base_seq = drums  # ベースになる音
total_sec = 60  # 生成したい曲の秒数
temperature = 1.2  # 曲のランダム度合い
base_end_time = 1.0  #ベース曲の終了時刻

# 生成器のオプションの設定
options = generator_pb2.GeneratorOptions()  # 生成器のオプション
options.args["temperature"].float_value = temperature  # ランダム度合い
options.generate_sections.add(
    start_time = base_end_time,  # 作曲開始時刻
    end_time = total_sec  # 作曲終了時刻
)
# 曲の生成
gen_seq = drums_rnn.generate(base_seq, options)

# 再生
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

###(4) 生成した音楽データを保存

In [9]:
# MIDIデータに変換し保存
note_seq.sequence_proto_to_midi_file(
    gen_seq, "magenta/drums_rnn1.mid"
)  

##《Peformance RNN で高度なピアノ演奏の作曲》
Peformance RNN で高度なピアノ演奏の作曲の作曲をしてみましょう。
Peformance RNN の詳細については、[こちら](https://github.com/magenta/magenta/tree/main/magenta/models/performance_rnn)を確認してください。

###(1) 基本となる音を与える
**もし、「《Polyphony RNN でバッハ風合唱曲の作曲》」を飛ばした場合には、「(1) ライブラリのインストール」を行ってから、以下を進めてください。**
<br>Peformance RNN では、楽器も指定することができます。

In [ ]:
from IPython.core.display import set_matplotlib_formats


# キーボードから音符を入力する関数
def base_music_create(base_music):
    start = 0.0
    total_time = 0.0
    # 音階とピッチの辞書
    scale_to_pich = {
        "ド": 60,
        "レ": 62,
        "ミ": 64,
        "ファ": 65,
        "ソ": 67,
        "ラ": 69,
        "シ": 71,
        "ド^": 72,
    }

    print("カタカナで「ド」～「ド^」を「,」区切りで入力してください。")
    print("【例】ファ,ファ,ファ,ソ,ラ,ラ,ラ,ラ,ソ,ファ,ソ,ラ,ファ,ファ,ド")
    onpu = input("音符＞ ")
    onpu = onpu.split(",")
    print("楽器を選んでください。")
    print("1 : ピアノ, 3 : ホンキートンクピアノ, 4 : エレクトリックピアノ, 6 : ハープシーコード")
    print("7 : クラヴィコード, 8 : チェレスタ, 9 : グロッケンシュピール, 10 : アンティークシンバル")
    print("11 : ビブラフォン, 12 : マリンバ, 13 : シロフォン, 14 : チャイム")
    print("15 : ダルシマー, 30 : エレキギター")
    pg = int(input("楽器＞ "))
    for on in onpu:
        if on in scale_to_pich:
            base_music.notes.add(
                pitch=scale_to_pich[on],
                start_time=start,
                end_time=start + 0.5,
                velocity=80,
                program=pg
            )
            start += 0.5
            total_time += 0.5

    # 所要時間の返却
    return total_time


import magenta
import note_seq
from note_seq.protobuf import music_pb2

# 基本の音を格納するbase_musicを作成
base_music = music_pb2.NoteSequence()

# 音符入力関数の呼び出し
total = base_music_create(base_music)

base_music.total_time = total  # 所要時間
base_music.tempos.add(qpm=75.0)  # 曲のテンポ（4分音符が1分間に75）

# 再生
note_seq.play_sequence(base_music, synth=note_seq.fluidsynth)  

###(2) 学習済みモデルの準備
Peformance RNN では、学習済みモデルには、以下が用意されています。この中から、「multiconditioned_performance_with_dynamics.mag.mag」を読み込み利用します。
- performance.mag
- performance_with_dynamics.mag
- performance_with_dynamics_and_modulo_encoding.mag
- density_conditioned_performance_with_dynamics.mag
- pitch_conditioned_performance_with_dynamics.mag
- multiconditioned_performance_with_dynamics.mag

なお、公式サイトには、
```
最初の 3 つのモデルは、異なるパフォーマンス表現を使用します。
最初のperformanceは音速を無視しますが、モデルは表現力豊かなタイミングでオン/オフ イベントを記録します。
performance_with_dynamicsモデルには、32 個のビンに量子化された速度の変化が含まれています。
このモデルは、 Vida Vakilotojarperformance_with_dynamics_and_modulo_encodingによって設計された代替エンコーディングを使用しており、
イベント値は単位円上の点にマッピングされます。
後者の 3 つのモデルは、それぞれ、目的のノート密度、目的のピッチ クラス分布、
またはその両方で条件付けされたパフォーマンスを生成できる条件付きモデルです。
```
とあります。

In [13]:
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# 学習済みモデルの初期化
note_seq.notebook_utils.download_bundle(  # 学習済みデータのダウンロード
    "multiconditioned_performance_with_dynamics.mag", "magenta/models/"
)
bundle = sequence_generator_bundle.read_bundle_file(  # 学習済みデータの読み込み
    "magenta/models/multiconditioned_performance_with_dynamics.mag"
)
generator_map = performance_sequence_generator.get_generator_map()   # 生成器のmapの作成
performance_rnn = generator_map["multiconditioned_performance_with_dynamics"](checkpoint=None, bundle=bundle)  # mapから生成器の設定
performance_rnn.initialize()  # 初期化

###(3) 自動作曲
上記で生成した生成器を使って作曲を行います。

In [ ]:
from note_seq.protobuf import generator_pb2

base_seq = base_music  # ベースになる音
total_sec = 120  # 生成したい曲の秒数（自由に変更可能）
temperature = 1.0  # 曲のランダム度合い（これを大きくするとランダム度合いが大きくなる）
base_end_time = base_music.total_time  #ベース曲の終了時刻

# 生成器のオプションの設定
options = generator_pb2.GeneratorOptions()  # 生成器のオプション
options.args["temperature"].float_value = temperature  # ランダム度合い
options.generate_sections.add(
    start_time = base_end_time,  # 作曲開始時刻
    end_time = total_sec  # 作曲終了時刻
)

# 曲の生成
gen_seq = performance_rnn.generate(base_seq, options)

# 再生
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

###(4) 生成した音楽データを保存

In [15]:
# MIDIデータに変換し保存
note_seq.sequence_proto_to_midi_file(
    gen_seq, "magenta/performance_rnn1.mid"
)  

##《演習》
1. Peformance RNN の学習済みモデルを変更する<br>Peformance RNN には4つの学習済みモデルが用意されています。それぞれダウンロードして自動作曲してみましょう。
2. 曲のテンポや長さの変更<br>曲のテンポや長さを自由に変更して曲を作成してみましょう。
3. ベースの音を変更<br>基本となる音を色々と入力して、どのような曲ができあがるか試してみましょう。
4. ベースの楽器を変更<br>Peformance RNN では楽器を指定することができます。いろいろな楽器を指定して、どのような曲ができあがるか試してみましょう。


##《参考文献》
- 斎藤 喜寛 「[Magentaで開発 AI作曲](https://www.ohmsha.co.jp/book/9784274227318/)」 オーム社  2021/07/20出版
- 吾妻 幸長  「[【Magenta+Colab】AIによる作曲を学ぼう！ -ディープラーニングで自動生成する音楽データ](https://www.udemy.com/course/ai-music/)」Udemy